In [28]:
# فراخوانی کتابخانه numpy
import numpy as np

# نمایش مرکز جدید خوشه
def getClusterRepresentative(inds, X):
    # میانگین از کلیه نمونه هایی که داخل یک خوشه افتادند
    rep = np.mean(X.iloc[inds])
    # بازگرداندن مرکز جدید محاسبه شده
    return rep

In [29]:
# فراخوانی کتابخانه numpy
import numpy as np
# تابع پیدا کردن نزدیک ترین خوشه 
def findClosestCluster(ii, labels, X):
    # در خط پایین کلیه برچسب های منحصر بفرد را که تاکنون تخصیص داده درون ارایه جدید می ریزد
    ulabels = np.unique(labels)
    # این شرط چک میکند که آیا عنصر اول لیست صفر هست یا خیر
    if (ulabels[0] == 0):
        # در صورتی که صفر بود آن را از لیست حذف میکند
        ulabels = ulabels[1:]
    # اینجا یک ارایه خالی جهت اضافه نمودن فاصله اقلیدسی کلیه نمونه های یک خوشه از مرکز جدید ایجا می شود
    x_ii_to_cluster = [];
    # یک حلقه که به تعداد برچسب های منحصر بفردی که تاکنون تخصیص داده تکرار می شود
    for lab in ulabels:
        # اینجا کلیه اندیس هایی که مربوط به هر برچسب خاص هستند در لیست برچسب پیدا می شوند
        inds = np.where( labels == lab )
        # در خط پایین مرکز جدید خوشه محاسبه می شود
        rep  = getClusterRepresentative( inds , X )
        # فاصله کلیه نمونه های هر خوشه از مرکز جدید هر خوشه محاسبه می شود.
        d = np.sqrt( np.dot(( X.iloc[ii].T - rep ) ,( X.iloc[ii].T - rep )) )
        # این فاصله ها به لیست پایین اضافه می شوند
        x_ii_to_cluster = np.append(x_ii_to_cluster,d)
    # در خط پایین کمینه لیست فاصله ها بدست می اید
    d_x_i_C_k = np.min(x_ii_to_cluster) 
    # اندیس عنصر کمینه بدست می اید
    mind = np.argmin(x_ii_to_cluster)
    # برچسب متناسب با عنصر کمینه بدست می اید
    k = ulabels[mind]
    # برچسب عنصر کیمنه و لیست فاصله ها به خروجی داده می شود
    return d_x_i_C_k,k

In [47]:


# فراخوانی کتابخانه numpy
import numpy as np
# تابع خوشه بندی که نمونه ها و مقادیر استانه تتا۱ و تتا۲ را می گیرد و برچسب های خروجی میدهد
def TTSAS(X, theta1, theta2):
    # تعداد نمونه ها
    N = X.shape[0]
    # تعداد ویژگی هر نمونه
    nFeatures = X.shape[1]
    # شمارنده برجسب ها
    m = 0
    # یک ارایه برای نگهداری اینکه ایا یک نمونه برچسب خورده یا خیر
    is_labeled = np.zeros(N)
    # برای نگهداری تغییرات گذشته
    prev_change = 0
    # برای نگهداری تغییرات فعلی
    cur_change = 0
    # ایا کلا تغییر وجود داشته
    exists_change = 0
    # یک لیست برای نگهداری برچسب های مربوط به نمونه ها
    labels = np.zeros(N)
    # شمارنده برای نگهداری تعداد دورها
    iteration = 0
    print('Runnig ...')
    # یک حلقه با شرط اینکه تا زمانیکه مجموع عناصر لیست کمتر از تعداد نمونه هاست
    while (np.sum(is_labeled) < N):
        # شمارنده دور اضافه می شود
        iteration = iteration + 1
        print(iteration)
        # چک می نماید که حلقه در اولین دور است
        first_in_while_loop = 1
        # یک حلقه بروی تمامی نمونه های ورودی
        for ii in range(N):
            # این شرط اولین دور حلقه وایل را چک میکند
            if (is_labeled[ii] == 0 and first_in_while_loop == 1 and exists_change == 0):
                # اگر شرط در ست یود
                # چک کننده اولین دور، صفر می شود
                first_in_while_loop = 0
                # شمارنده برچسب یکی اضافه می شود
                m = m + 1
                # برچسب نمونه آي م برابر با مقدار ام می شود
                labels[ii] = m
                # در ماتریس برچسب خورده مقدار نمونه ای م یک می شود
                # که نشان دهنده این است که این نمونه برچسب خورده یا خیر
                is_labeled[ii] = 1
                # پارامتر تعداد تغییرات جاری اضافه می شود
                cur_change = cur_change + 1
            # این شرط برای دورهای دوم به بعد حلقه وایل چک می شود
            # اگر نمونه ای م برچسب نخورده بود وارد شرط می شویم
            elif (is_labeled[ii] == 0):
                # در این مرحله نزدیک ترین خوشه به نمونه محاسبه می شود مطابق تا بع بالا
                d_x_i_C_k, k = findClosestCluster(ii, labels, X)
                #  این شرط چک میکند که اگر فاصله نمونه تا خوشه کمتر از استانه اول بود
                if (d_x_i_C_k < theta1):
                    # آنگاه برچسب نزدیک ترین خوشه به نمونه مورد نظر داده می شود
                    labels[ii] = k
                    # درون بردار برچسب خورده عنصر مربوط به نمونه ای م یک می شود
                    is_labeled[ii] = 1
                    # تغییرات جاری یکی اضافه می شود
                    cur_change = cur_change + 1
                # اگر فاصله بین نمونه ای م و خوشه های محاصبه شد بیشتر از استانه دوم بود
                elif (d_x_i_C_k > theta2):
                    # انگاه به این نمونه یک برچسب جدید اختاصی میدهیم
                    m = m + 1
                    # درون لیست برچسب های برچسب نمونه ای م ست می شود
                    labels[ii] = m
                    # درون لیست برچسب خورده نمونه ای م به حالت برچسب خورده تغییر می کند
                    is_labeled[ii] = 1
                    # تغییرات جاری اضافه می شود
                    cur_change = cur_change + 1
            # در این شرط چک می شود که ایا نمونه ای م برچسب خورده است در صورتی که خورده بود
            elif (is_labeled[ii] == 1):
                # فقط تغییرات جاری اضافه می شود
                cur_change = cur_change + 1
            else:
                # اگر نمونه مشکل داشت پیغام پایین چاپ می شود
                print('should never get here\n')
        # تفاوت وضعیت جاری به وضعیت قبلی محاسبه می شود و در متغیر تغییرات موجود ریحته می شود
        exists_change = np.abs( cur_change - prev_change ) 
        # وضعیت فعلی درون وضعیت قبلی ریحته می شود
        prev_change   = cur_change
        # وضعیت جاری ریست می شود
        cur_change    = 0
    # برچسب ها برگردانده می شوند.
    return labels
  
    
                
      

In [48]:
import pandas as pd

In [49]:
X = pd.read_csv('data.txt',sep='\t')

In [50]:
theta1 = 0.5 
theta2 = 0.75 
labs = TTSAS( X, theta1, theta2 );


Runnig ...
1
2
3
4
5
6
7
8


In [51]:
print labs

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  2.  2.  3.  2.  2.  2.  3.  3.  3.  2.
  3.  2.  3.  2.  2.  3.  2.  2.  3.  2.  2.  2.  2.  2.  2.  2.  3.  3.
  3.  3.  8.  2.  2.  2.  2.  3.  3.  2.  2.  3.  3.  2.  2.  2.  2.  7.
  2.  4.  5.  5.  4.  6.  6.  4.  4.  5.  5.  5.  5.  5.  5.  5.  6.  6.
  9.  4.  6.  5.  4.  4.  5.  4. 10.  4.  6.  4.  9. 10.  4.  4.  5.  5.
  5.  4.  5.  5.  4.  4.  5.  5.  5.  5.  5.]
